# Importing Libraries

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
import numpy as np

In [2]:
# Connecting With Google BigQuery

from google.oauth2 import service_account #For GCP Account connection
from google.cloud import bigquery
#Setting Up Project ID
project_id = "digitalmarketing-a2-247505"
#Setting up the credential file
cred = service_account.Credentials.from_service_account_file('digitalMarketing-a2-470cdee74823.json')

In [3]:
#Setting up a BigQuery Client:
client = bigquery.Client(project=project_id, credentials=cred)

In [4]:
# SQL Statement to be ran on BigQuery to fetch data

sql = """
SELECT t.*
FROM `digitalmarketing-a2-247505.digitalMarketing_Assignment2.Transactions` t
"""

In [5]:
data = client.query(sql).to_dataframe()

In [6]:
data.head()

,InvoiceNo,Product_ID,InvoiceDate,Customer_ID,Promotion_ID,Total_Amount_Per_Month,Number_of_months_bought,Total_Amount
0,540672,7,2018-01-10,15281,3,5.0915,1,5.0915
1,540249,15,2018-01-05,15298,3,5.0915,2,10.1830
2,541012,7,2018-01-13,17019,3,5.0915,2,10.1830
3,546861,7,2018-03-17,14911,3,5.0915,2,10.1830
4,561196,7,2018-07-25,14342,5,5.0915,2,10.1830


In [7]:
data.columns= ['InvoiceNo','ProductCode','InvoiceDate','CustomerID','Promotion_ID','UnitPrice','Quantity','Total_Amount']

In [8]:
data.head()

,InvoiceNo,ProductCode,InvoiceDate,CustomerID,Promotion_ID,UnitPrice,Quantity,Total_Amount
0,540672,7,2018-01-10,15281,3,5.0915,1,5.0915
1,540249,15,2018-01-05,15298,3,5.0915,2,10.1830
2,541012,7,2018-01-13,17019,3,5.0915,2,10.1830
3,546861,7,2018-03-17,14911,3,5.0915,2,10.1830
4,561196,7,2018-07-25,14342,5,5.0915,2,10.1830


In [9]:
filtered_data=data[['CustomerID']].drop_duplicates()

In [10]:
#Top ten country's customer
#filtered_data.Country.value_counts()[:10].plot(kind='bar')

In [11]:
#uk_data=data[data.Country=='United Kingdom']
#uk_data.info()

In [12]:
#uk_data.describe()

In [13]:
data = data[(data['Quantity']>0)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337 entries, 0 to 1336
Data columns (total 8 columns):
InvoiceNo       1337 non-null int64
ProductCode     1337 non-null int64
InvoiceDate     1337 non-null object
CustomerID      1337 non-null int64
Promotion_ID    1337 non-null int64
UnitPrice       1337 non-null float64
Quantity        1337 non-null int64
Total_Amount    1337 non-null float64
dtypes: float64(2), int64(5), object(1)
memory usage: 94.0+ KB


In [14]:
data=data[['CustomerID','InvoiceDate','InvoiceNo','Quantity','UnitPrice']]

In [15]:
#Calulate total purchase
data['TotalPurchase'] = data['Quantity'] * data['UnitPrice']

In [16]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [17]:
data_group=data.groupby('CustomerID').agg({'InvoiceDate': lambda date: (date.max() - date.min()).days,
                                        'InvoiceNo': lambda num: len(num),
                                        'Quantity': lambda quant: quant.sum(),
                                        'TotalPurchase': lambda price: price.sum()})

In [18]:
data_group.head()

,InvoiceDate,InvoiceNo,Quantity,TotalPurchase
CustomerID,,,,
12347,0,1,8,119.9200
12370,0,1,3,38.2245
12377,0,2,19,220.8385
12383,0,1,1,8.9910
12408,0,1,5,59.9600


In [19]:
data_group.columns=['num_days','num_transactions','num_units','spent_money']
data_group.head()

,num_days,num_transactions,num_units,spent_money
CustomerID,,,,
12347,0,1,8,119.9200
12370,0,1,3,38.2245
12377,0,2,19,220.8385
12383,0,1,1,8.9910
12408,0,1,5,59.9600


In [20]:
# Average Order Value
data_group['avg_order_value']=data_group['spent_money']/data_group['num_transactions']

In [21]:
data_group.head()

,num_days,num_transactions,num_units,spent_money,avg_order_value
CustomerID,,,,,
12347,0,1,8,119.9200,119.92000
12370,0,1,3,38.2245,38.22450
12377,0,2,19,220.8385,110.41925
12383,0,1,1,8.9910,8.99100
12408,0,1,5,59.9600,59.96000


In [22]:
purchase_frequency=sum(data_group['num_transactions'])/data_group.shape[0]

In [23]:
# Repeat Rate
repeat_rate=data_group[data_group.num_transactions > 1].shape[0]/data_group.shape[0]

In [24]:
#Churn Rate
churn_rate=1-repeat_rate

In [25]:
purchase_frequency,repeat_rate,churn_rate

(2.125596184419714, 0.4260731319554849, 0.5739268680445151)

Let's assume our business has approx 5% profit on the total sale.

In [26]:
# Profit Margin
data_group['profit_margin']=data_group['spent_money']*0.05
data_group.head()

,num_days,num_transactions,num_units,spent_money,avg_order_value,profit_margin
CustomerID,,,,,,
12347,0,1,8,119.9200,119.92000,5.996000
12370,0,1,3,38.2245,38.22450,1.911225
12377,0,2,19,220.8385,110.41925,11.041925
12383,0,1,1,8.9910,8.99100,0.449550
12408,0,1,5,59.9600,59.96000,2.998000


In [27]:
# Customer Value
data_group['CLV']=(data_group['avg_order_value']*purchase_frequency)/churn_rate
#Customer Lifetime Value
data_group['cust_lifetime_value']=data_group['CLV']*data_group['profit_margin']
data_group.head()

,num_days,num_transactions,num_units,spent_money,avg_order_value,profit_margin,CLV,cust_lifetime_value
CustomerID,,,,,,,,
12347,0,1,8,119.9200,119.92000,5.996000,444.135845,2663.038526
12370,0,1,3,38.2245,38.22450,1.911225,141.568301,270.568875
12377,0,2,19,220.8385,110.41925,11.041925,408.948857,4515.582604
12383,0,1,1,8.9910,8.99100,0.449550,33.299078,14.969600
12408,0,1,5,59.9600,59.96000,2.998000,222.067922,665.759631


In [28]:
data.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPurchase
0,15281,2018-01-10,540672,1,5.0915,5.0915
1,15298,2018-01-05,540249,2,5.0915,10.1830
2,17019,2018-01-13,541012,2,5.0915,10.1830
3,14911,2018-03-17,546861,2,5.0915,10.1830
4,14342,2018-07-25,561196,2,5.0915,10.1830


In [29]:
data['month_yr'] = data['InvoiceDate'].apply(lambda x: x.strftime('%b-%Y'))

data.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPurchase,month_yr
0,15281,2018-01-10,540672,1,5.0915,5.0915,Jan-2018
1,15298,2018-01-05,540249,2,5.0915,10.1830,Jan-2018
2,17019,2018-01-13,541012,2,5.0915,10.1830,Jan-2018
3,14911,2018-03-17,546861,2,5.0915,10.1830,Mar-2018
4,14342,2018-07-25,561196,2,5.0915,10.1830,Jul-2018


In [30]:
sale=data.pivot_table(index=['CustomerID'],columns=['month_yr'],values='TotalPurchase',aggfunc='sum',fill_value=0).reset_index()

sale.head()

month_yr,CustomerID,Apr-2018,Aug-2018,Dec-2018,Feb-2018,Jan-2018,Jul-2018,Jun-2018,Mar-2018,May-2018,Nov-2018,Oct-2018,Sep-2018
0,12347,0.0,0.0,0.000,0.0,0.0000,0.0,119.92,0.0000,0.0,0.00,0.0,0.0
1,12370,0.0,0.0,0.000,0.0,0.0000,0.0,0.00,38.2245,0.0,0.00,0.0,0.0
2,12377,0.0,0.0,0.000,0.0,220.8385,0.0,0.00,0.0000,0.0,0.00,0.0,0.0
3,12383,0.0,0.0,8.991,0.0,0.0000,0.0,0.00,0.0000,0.0,0.00,0.0,0.0
4,12408,0.0,0.0,0.000,0.0,0.0000,0.0,0.00,0.0000,0.0,59.96,0.0,0.0


In [31]:
sale['CLV']=sale.iloc[:,2:].sum(axis=1)

sale.head()

month_yr,CustomerID,Apr-2018,Aug-2018,Dec-2018,Feb-2018,Jan-2018,Jul-2018,Jun-2018,Mar-2018,May-2018,Nov-2018,Oct-2018,Sep-2018,CLV
0,12347,0.0,0.0,0.000,0.0,0.0000,0.0,119.92,0.0000,0.0,0.00,0.0,0.0,119.9200
1,12370,0.0,0.0,0.000,0.0,0.0000,0.0,0.00,38.2245,0.0,0.00,0.0,0.0,38.2245
2,12377,0.0,0.0,0.000,0.0,220.8385,0.0,0.00,0.0000,0.0,0.00,0.0,0.0,220.8385
3,12383,0.0,0.0,8.991,0.0,0.0000,0.0,0.00,0.0000,0.0,0.00,0.0,0.0,8.9910
4,12408,0.0,0.0,0.000,0.0,0.0000,0.0,0.00,0.0000,0.0,59.96,0.0,0.0,59.9600


In [32]:
X=sale[['Dec-2018','Nov-2018', 'Oct-2018','Sep-2018','Aug-2018','Jul-2018']]
y=sale[['CLV']]

In [33]:
#split training set and test set
#from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [34]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

# make predictions on the testing set
y_pred = linreg.predict(X_test)

In [35]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

[20.91222556]
[[1.30623351 1.00072835 1.4254322  0.70499086 1.81467571 1.3841365 ]]


In [36]:
from sklearn import metrics

# compute the R Square for model
print("R-Square:",metrics.r2_score(y_test, y_pred))

R-Square: 0.7981868397165358


In [37]:
# calculate MAE using scikit-learn
print("MAE:",metrics.mean_absolute_error(y_test,y_pred))

#calculate mean squared error
print("MSE",metrics.mean_squared_error(y_test, y_pred))
# compute the RMSE of our predictions
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MAE: 43.52727669997623
MSE 4176.814535872493
RMSE: 64.62827969142064
